# DynamoDB Callback

The notebook showcases how to use the [StreamingDynamoDBCallbackHandler](./slangchain/callbacks/streaming_aws_ddb.py) class. The functionality streams callbacks to a AWS DyanmoDB table

[LangChain](https://github.com/hwchase17/langchain) provides a callback system that allows you to hook into the various stages of your LLM application. This is useful for logging, [monitoring](https://python.langchain.com/en/latest/tracing.html), [streaming](https://python.langchain.com/en/latest/modules/models/llms/examples/streaming_llm.html), and other tasks.

## Prerequisites

- [An AWS account](https://aws.amazon.com/account/sign-up)
- [AWS credentials setup](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html)

## AWS Setup

Create a test DyanmoDB table for this example.

In [1]:
import boto3

table_name = "langchain_callback_datastore"

ddb_resource = boto3.resource("dynamodb")
ddb_resource.create_table(
  TableName = table_name,
  KeySchema=[
    {'AttributeName': 'partition_key', 'KeyType': 'HASH'},  # Partition key
    {'AttributeName': 'sort_key', 'KeyType': 'RANGE'}  # Sort key
  ],
  AttributeDefinitions=[
    {'AttributeName': 'partition_key', 'AttributeType': 'S'},
    {'AttributeName': 'sort_key', 'AttributeType': 'S'}
  ],
  ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10}
)


dynamodb.Table(name='langchain_callback_datastore')

## Example

Create an on agent action callback (You might have to wait until the DynamoDB table is created before running the below code)

In [2]:
from uuid import uuid4
from langchain.schema import AgentAction
from slangchain.callbacks.streaming_aws_ddb import StreamingDynamoDBCallbackHandler

# set a partition key value
partition_key_value = str(uuid4())
# set a sort key value
sort_key_prefix_value = str(uuid4())

handler = StreamingDynamoDBCallbackHandler(
    ddb_resource=ddb_resource,
    ddb_table_name=table_name,
    partition_key_value=partition_key_value,
    sort_key_prefix_value=sort_key_prefix_value,
)

handler.on_agent_action(AgentAction("tool", "tool_input", "log"))

Validate that the item has been created in the DynamoDB table

In [3]:
from slangchain.aws.dynamodb.base import DDBAppLoader

ddb_loader = DDBAppLoader(
  ddb_resource=ddb_resource,
  table_name=table_name,
  partition_key_value=partition_key_value,
  sort_key_value=sort_key_prefix_value)

ddb_loader.load()

DDBPayloadList(__root__=[DDBPayload(partition_key='ca89cb3f-4a3d-4a33-8653-571e1cb4f594', sort_key='9669ebb8-532c-4178-9b58-c6bb441e20a9#0f6e2a0a-29a3-44ae-aba4-5d2496a696d5#1682567010.5875661#AGENT_ACTION#000001', record_type='AGENT_ACTION', timestamp=1682567010.5875661, sequence_number=1, payload={'message': {'tool_input': 'tool_input', 'log': 'log', 'tool': 'tool'}, 'arguments': {}})])

Cleaning up AWS resources

In [4]:
client = boto3.client("dynamodb")
client.delete_table(
    TableName=table_name
)

{'TableDescription': {'TableName': 'langchain_callback_datastore',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ap-southeast-2:734145044054:table/langchain_callback_datastore',
  'TableId': 'a9da0b0c-eb91-4b46-b0b7-a5967f9cdeeb'},
 'ResponseMetadata': {'RequestId': 'J67GHDV1KEUUACPPE47DUJGB9NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 27 Apr 2023 03:43:37 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '399',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'J67GHDV1KEUUACPPE47DUJGB9NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3489350072'},
  'RetryAttempts': 0}}